<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/PIC_Constru%C3%A7%C3%A3o_de_modelos_de_detec%C3%A7%C3%A3o_autom%C3%A1tica_de_linguagem_ofensiva_e_ou_de_%C3%B3dio_em_l%C3%ADngua_portuguesa_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas básicas

In [28]:
 !pip install plotly --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
print()

In [30]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
#Importing needed libraries 

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

#NLTK tools for text processing 

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS

#Modeling packages 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

# Base de dados Artigo 1 (ToLD-BR)

---



In [31]:
base_toldbr = pd.read_csv('/content/drive/MyDrive/Dataset/ToLD-BR.csv') #importação da base de dados

In [32]:
base_toldbr #Mostrando os dados importados

,text,homophobia,obscene,insult,racism,misogyny,xenophobia
0,Meu nivel de amizade com isis é ela ter meu in...,0.0,0.0,2.0,0.0,0.0,0.0
1,"rt @user @user o cara adultera dados, que fora...",0.0,0.0,1.0,0.0,0.0,0.0
2,@user @user @user o cara só é simplesmente o m...,0.0,2.0,1.0,0.0,0.0,0.0
3,eu to chorando vei vsf e eu nem staneio izone ...,0.0,1.0,0.0,0.0,0.0,0.0
4,Eleitor do Bolsonaro é tão ignorante q não per...,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
20995,@user faz favor vai carai,0.0,0.0,0.0,0.0,0.0,0.0
20996,só queria conhecer alguém que não conhece o he...,1.0,0.0,0.0,0.0,0.0,0.0
20997,"vcs militam na hora errada em cima de memes, p...",0.0,0.0,0.0,0.0,0.0,0.0
20998,@user porra any eu tava c dor de cabeca e fui ...,0.0,0.0,0.0,0.0,0.0,0.0


**Verificando o tamanho do datasets**

In [33]:
base_toldbr.shape #linhas x colunas

(21000, 7)

**Conferindo o total de textos por categorias**

In [34]:
base_toldbr['homophobia'].value_counts() 

0.0    20656
1.0      168
2.0      102
3.0       74
Name: homophobia, dtype: int64

In [35]:
base_toldbr['obscene'].value_counts()

0.0    14348
1.0     4249
2.0     1791
3.0      612
Name: obscene, dtype: int64

In [36]:
base_toldbr['insult'].value_counts()

0.0    16615
1.0     2516
2.0     1352
3.0      517
Name: insult, dtype: int64

In [37]:
base_toldbr['racism'].value_counts()

0.0    20862
1.0      105
2.0       27
3.0        6
Name: racism, dtype: int64

In [38]:
base_toldbr['misogyny'].value_counts()

0.0    20537
1.0      330
2.0      104
3.0       29
Name: misogyny, dtype: int64

In [39]:
base_toldbr['xenophobia'].value_counts()

0.0    20849
1.0      109
2.0       27
3.0       15
Name: xenophobia, dtype: int64

Verificando valores nulos e tipos de dados

In [40]:
base_toldbr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21000 entries, 0 to 20999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   text        21000 non-null  object 
 1   homophobia  21000 non-null  float64
 2   obscene     21000 non-null  float64
 3   insult      21000 non-null  float64
 4   racism      21000 non-null  float64
 5   misogyny    21000 non-null  float64
 6   xenophobia  21000 non-null  float64
dtypes: float64(6), object(1)
memory usage: 1.1+ MB


**Como verificado logo acima não há valores NULL**

In [41]:
base_toldbr.isnull().sum()

text          0
homophobia    0
obscene       0
insult        0
racism        0
misogyny      0
xenophobia    0
dtype: int64

**Donwload dos pacotes que permitirão fazer os trabalhos de PLN**

In [42]:
import nltk 
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

**Limpeza dos dados**

**O que é Stop Words?**

Stop words (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi.

In [43]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_toldbr['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  stop = ['user', 'rt', 'https']
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))
  

    


In [44]:
print(np.std([1,2,3,4,5]))

1.4142135623730951


Ranque das palavras únicas com base na frequência de cada palavra


In [45]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


**Rank de palavras com maior frequência sem Stopword** 

In [46]:
important_unigrams[:5]

,0
co,3619
pra,2999
porra,1941
pqp,1726
caralho,1688


**Rank de palavras com maior frequência com Stopword** 

In [47]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_toldbr['text'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [48]:
important_unigrams[:5]

,0
user,13270
que,8045
de,7814
eu,5644
não,3875


**Verificando a quantidade de palavras na base (com e sem Stopwords)**


In [49]:
i = 0
for texto in comments:
  for palavras in texto:
      i = i + 1 

k = 0
for texto in base_toldbr['text']:
  for palavras in texto:
      k = k + 1 

print(f'Quantidade de palavras com Stopwords: {k} palavras')
print(f'Quantidade de palavras sem Stopwords: {i} palavras')

Quantidade de palavras com Stopwords: 1821116 palavras
Quantidade de palavras sem Stopwords: 1230392 palavras


In [3]:
# wordfreq =[]
lista_quantidade_palavras =[]
for texto in base_toldbr['text']:
  test = texto.split(' ')
  print(test)
  print(len(test))
  exit()
  lista_quantidade_palavras.append(len(test))


# lista = 'salve galerinha bonita'

# wordlist = lista.split(' ')
# print(len(wordlist))


NameError: ignored

In [2]:
print(lista_quantidade_palavras[-1])
print(lista_quantidade_palavras[0])

NameError: ignored


# Base de dados Artigo 2 (HateBR)

In [52]:
base_hatebr = pd.read_csv('/content/drive/MyDrive/Dataset/HateBR.csv')

In [53]:
base_hatebr

,instagram_comments,offensive_language,offensiveness_levels,hate_speech
0,este lixo ...,1,1,-1
1,Mais um lixo,1,1,-1
2,Essa nao tem vergonha na cara!!,1,2,-1
3,Essa mulher é doente.pilantra!,1,3,-1
4,Comunista safada...,1,2,"5,8"
...,...,...,...,...
6995,Time perdendo a credibilidade,0,0,0
6996,Siga em frente Presidente Bolsonaro! Afinal o ...,0,0,0
6997,Tantas coisas importantes para resolver e fica...,0,0,0
6998,"A TIME escolhe quem eles quiser, isso não sign...",0,0,0


**Verificando o tamanho da Base**

In [54]:
base_hatebr.shape #linhas x colunas

(7000, 4)

**Conferindo o total de textos por categorias**

Com o código abaixo podemos ver que os dados estão balanceados(número de classificação são iguais)

In [55]:
base_hatebr['offensive_language'].value_counts() 

1    3500
0    3500
Name: offensive_language, dtype: int64

In [56]:
# np.unique(base_hatebr['offensiveness_levels'], return_counts= True)
base_hatebr['offensiveness_levels'].value_counts() 

0    3500
2    1440
1    1281
3     779
Name: offensiveness_levels, dtype: int64

In [57]:
# np.unique(base_hatebr['hate_speech'], return_counts= True)
base_hatebr['hate_speech'].value_counts(ascending = True)  #verificar certinho do que se trata estes dados

9         1
5,7       1
3,5       1
4,6       2
1         2
2,5       4
3,8       6
6         6
5,8      11
4        15
3        20
2        28
7        46
8        80
5       479
-1     2798
0      3500
Name: hate_speech, dtype: int64

In [58]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_hatebr ['instagram_comments']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))
  

In [59]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [60]:
important_unigrams[:5]

,0
presidente,458
est,440
pra,437
brasil,389
ns,380


In [61]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_hatebr ['instagram_comments'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [62]:
important_unigrams[:5]

,0
que,2434
de,2069
não,1264
do,1014
um,782


**Verificando a quantidade de palavras na base (com e sem Stopwords)**


In [63]:
i = 0
for texto in comments:
  for palavras in texto:
      i = i + 1 

k = 0
for texto in base_hatebr['instagram_comments']:
  for palavras in texto:
      k = k + 1 


print(f'Quantidade de palavras com Stopwords: {k} palavras')
print(f'Quantidade de palavras sem Stopwords: {i} palavras')

Quantidade de palavras com Stopwords: 478170 palavras
Quantidade de palavras sem Stopwords: 336848 palavras


# Base de dados Artigo 3 (OffComBR2 e OffComBR3)

In [64]:
base_offcombr2 = pd.read_csv('/content/drive/MyDrive/Dataset/OffComBR2.csv')

In [65]:
base_offcombr2

,Class,Texto
0,yes,Votaram no PEZAO Agora tomem no CZAO'
1,no,cuidado com a poupanca pessoal Lembram o que a...
2,no,Sabe o que eu acho engracado os nossos governa...
3,yes,os cariocas tem o que merecem um pessoal que s...
4,no,Podiam retirar dos lucros dos bancos '
...,...,...
1245,yes,LIXO DO CRL'
1246,yes,ela esta fazendo isso para aparecer Nao e gay ...
1247,yes,Volta pra jaula sua macaca vou pegar meu xicot...
1248,yes,Achei que a macaca vivia apenas na floresta ou...


In [66]:
# np.unique(base_offcombr2['Class'], return_counts= True)
base_offcombr2['Class'].value_counts()

no     831
yes    419
Name: Class, dtype: int64

In [67]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_offcombr2 ['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))


In [68]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [69]:
important_unigrams[:5]

,0
nao,340
vai,131
so,107
pra,97
ja,75


In [70]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_offcombr2 ['Texto'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [71]:
important_unigrams[:5]

,0
de,474
que,462
nao,340
do,212
da,160


In [72]:
i = 0
for texto in comments:
  for palavras in texto:
      i = i + 1 

k = 0
for texto in base_offcombr2 ['Texto']:
  for palavras in texto:
      k = k + 1 

print(f'Quantidade de palavras com Stopwords: {k} palavras')
print(f'Quantidade de palavras sem Stopwords: {i} palavras')



Quantidade de palavras com Stopwords: 91949 palavras
Quantidade de palavras sem Stopwords: 68829 palavras


In [73]:
base_offcombr3 = pd.read_csv('/content/drive/MyDrive/Dataset/OffComBR3.csv')

In [74]:
base_offcombr3

,Class,Texto
0,yes,Votaram no PEZAO Agora tomem no CZAO'
1,no,cuidado com a poupanca pessoal Lembram o que a...
2,no,Sabe o que eu acho engracado os nossos governa...
3,no,Podiam retirar dos lucros dos bancos '
4,no,CADE O GALVAO PRA NARRAR AGORA FALIIIIUUUUUU...
...,...,...
1028,no,Cruz so tem agilidade mesmo poder de nocaute z...
1029,no,Meus caros amigos enigmaticosNao deveriam comp...
1030,yes,Ele chamava pra atras da escola e sentava nos ...
1031,no,Jhalim Rabei ate fiquei assustado comecei a le...


In [75]:
# np.unique(base_offcombr2['Class'], return_counts= True)
base_offcombr3['Class'].value_counts()

no     831
yes    202
Name: Class, dtype: int64

In [76]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_offcombr3 ['Texto']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))


In [77]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [78]:
important_unigrams[:5]

,0
nao,293
vai,99
so,90
pra,81
ja,68


In [79]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_offcombr3 ['Texto'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [80]:
important_unigrams[:5]

,0
que,403
de,399
nao,293
do,177
da,138


# Base de dados Artigo 4 (base_hate_speech_binary e hate_speech_hierarchical)

In [81]:
base_hate_speech_binary = pd.read_csv('/content/drive/MyDrive/Dataset/2019-05-28_portuguese_hate_speech_binary_classification.csv')

In [82]:
base_hate_speech_binary

,text,hatespeech_comb,hatespeech_G1,annotator_G1,hatespeech_G2,annotator_G2,hatespeech_G3,annotator_G3
0,@__andrea__b \nO cara vive em outro mundo\nNão...,1,1,A,1.0,V,0,E
1,@_carmeloneto Estes incompetentes não cuidam n...,0,1,D,0.0,V,0,C
2,@_carmeloneto \nOs 'cumpanhero' quebraram toda...,0,1,A,0.0,B,0,E
3,@_GlitteryKisses é isso não conseguem pensar n...,0,0,C,0.0,V,0,D
4,@_iglira bom dia macaco branco haha,1,0,A,1.0,I,1,E
...,...,...,...,...,...,...,...,...
5665,@zecarlosantos2 é o unico que nao se corrompe....,0,1,C,0.0,B,0,A
5666,"@zqkitowz sei das cotas, mas não sabia disso, ...",1,1,D,1.0,It,0,A
5667,"@zqkitowz sim, a maioria do eleitorado é mulhe...",0,0,C,0.0,V,0,C
5668,"@zurcju seguir no tt é facíl, apresentar as am...",1,1,C,1.0,S,0,A


In [83]:
base_hate_speech_binary['hatespeech_comb'].value_counts() 

0    3882
1    1788
Name: hatespeech_comb, dtype: int64

In [84]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in base_hate_speech_binary['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))

In [85]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [86]:
important_unigrams[:5]

,0
co,2424
mulher,516
pra,491
dia,426
vai,251


In [87]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(base_hate_speech_binary['text'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [88]:
important_unigrams[:5]

,0
de,2457
co,2421
https,2338
que,2141
não,1644


In [89]:
i = 0
for texto in comments:
  for palavras in texto:
      i = i + 1 

k = 0
for texto in base_hate_speech_binary['text']:
  for palavras in texto:
      k = k + 1 

print(f'Quantidade de palavras com Stopwords: {k} palavras')
print(f'Quantidade de palavras sem Stopwords: {i} palavras')



Quantidade de palavras com Stopwords: 572821 palavras
Quantidade de palavras sem Stopwords: 411091 palavras


In [90]:
hate_speech_hierarchical = pd.read_csv('/content/drive/MyDrive/Dataset/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv')

In [91]:
hate_speech_hierarchical

,text,Hate.speech,Sexism,Body,Racism,Ideology,Homophobia,Origin,Religion,Health,...,Thin.women,Arabic,East.europeans,Africans,South.Americans,Brazilians,Migrants,Homossexuals,Thin.people,Ageing
0,"""não come mel, morde marimbondo""",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"não tem pinto, tem orgulho !",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Não vê essa merda de Crepúsculo! Pra isso temo...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"não da tapa na bundinha, da cotovelada nas cos...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,o diminutivo INHO não acompanha a trajetória d...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5663,Na minha sala só tem viado e sapatão e a cois...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5664,PARABENS SAPATÃO SDDS @attomiter https://t.co/...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5665,RT @toquedeveludo: Agora um poema:\nEu sou sap...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5666,O mundo das sapatao é mais ligado do que eu im...,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [92]:
comments = [] 
stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

for words in hate_speech_hierarchical['text']:
  only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
  tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
  lower_case = [l.lower() for l in tokens] # converter all letters to low case 
  filtered_result = list(filter(lambda l: l not in stop_words and  l not in stop, lower_case)) # Remove stopwords from the comments
  comments.append(' '.join(filtered_result))

In [93]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform(comments)
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [94]:
important_unigrams[:5]

,0
co,2425
mulher,516
pra,491
dia,420
vai,251


In [95]:
#using CountVectorizer to get the most important unigrams
co = CountVectorizer(ngram_range=(1,1))
count = co.fit_transform( hate_speech_hierarchical['text'])
important_unigrams =pd.DataFrame(count.sum(axis=0), columns=co.get_feature_names()).T.sort_values(0, ascending = False)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [96]:
important_unigrams

,0
de,2457
co,2422
https,2395
que,2140
não,1647
...,...
imortal,1
imp,1
impacto,1
imparcial,1
